# Snakes and Ladders

Solution and Implementation by Augusto Ferreira
Date: 2023/12/07

You and your friend are playing your old Snakes and Ladders board game. Each player begins on square 1 and takes turns rolling a fair 6-sided die. The player moves the number of spaces indicated on the die. If you land at the bottom of a ladder, you automatically move to the square at the top of the ladder. Conversely, if you land on a snake head, then you fall to the square at the snake's tail. The winner is the first person to make it onto or past the last square.

Being interested in analytics, you decide to run simulations of 10,000 games to understand your odds of winning under different scenarios. Consider each scenario independent of the other scenarios.

Please answer the following questions using any means available. We would like you to keep a record of your work and walk us through your solution. We are primarily interested in your approach and code, rather than the final answers.

![Game Board](game_board.png "Game Board")

## Code
The implementation for classes and the game itself are available in this repository. You can check the in the Python files.

## Imports and execution
We only need to import a function to process the input file, which contains the coordinates for the snakes and for the ladders. We also need to import the function that plays the game and return the winner.

In [1]:
from main import process_input
from Game import play_round
from Game import play_round_immunity

In [2]:
from Player import Player

p1 = Player("Ale")
p2 = Player("Augusto")

players = [p1, p2]

moves, board = process_input("input0.json")

runs = 10000

## Questions

___
### Question 1
In a two person game, what is the probability that the player who starts the game wins?

In [3]:
winners = []
for _ in range(0, runs):
    winner, __ = play_round(players, board, moves)

    # Save the winner's name
    winners.append(winner.name)

    # Don't forget to reset the player current cell before playing again
    [p.reset() for p in players]

for p in players:
    print(p.name, winners.count(p.name) / len(winners))

Ale 0.5181
Augusto 0.4819


Considering Ale as the first player, the probability of her wins is about 53%.

___
### Question 2
On average, how many snakes are landed on in each game?

In [4]:
from MoveType import MoveType

average_sakes = 0
for _ in range(0, runs):
    play_round(players, board, moves)

    # Save all the cell lands
    lands = []
    for player in players:
        for move in player.history:
            lands.append(move)
            
    # Using an enum to count is faster than a string comparison
    average_sakes += lands.count(MoveType.SNAKE)

    # Don't forget to reset the player current cell before playing again
    [p.reset() for p in players]

print("Average snakes: ", float(average_sakes / runs))

Average snakes:  3.1899


Answer: The average snakes landed in the game is 3.2.

___
### Question 3
If each time a player landed on a ladder and there was only a 50% chance they could take it, what is the average number of rolls needed to complete a game?

In [5]:
total_rolls = 0
for _ in range(0, runs):
    __, rolls = play_round(players, board, moves, consider_probability=True)
    total_rolls += rolls

    # Don't forget to reset the player current cell before playing again
    [p.reset() for p in players]

print("Average rolls: ", float(total_rolls / runs))

Average rolls:  11.5224


Answer: the average rolls would increase from 9.8 to 11.5.

___
### Question 4
Starting with the base game, you decide you want the game to have approximately fair odds. You do this by changing the square that Player 2 starts on. Which square for Player 2’s start position gives the closest to equal odds for both players?

In [6]:
import pandas as pd

results = {
    "Square": [],
    "Player1": [],
    "Player2": [],
}

for initial_square in range(1, board.last_cell):
    results["Square"].append(initial_square)
    winners = []
    for _ in range(0, runs):
        # Start p2 at the i_th square
        p2.current = initial_square

        winner, __ = play_round(players, board, moves)

        # Save the winner's name
        winners.append(winner.name)

        # Don't forget to reset the player current cell before playing again
        [p.reset() for p in players]

    results["Player1"].append(winners.count(p1.name) / len(winners))
    results["Player2"].append(winners.count(p2.name) / len(winners))

In [9]:
df = pd.DataFrame(results)

from IPython.display import display, HTML

display(HTML(df.to_html(index=False)))

Square,Player1,Player2
1,0.5274,0.4726
2,0.5150,0.4850
3,0.5390,0.4610
4,0.5263,0.4737
5,0.5213,0.4787
6,0.5175,0.4825
7,0.5044,0.4956
8,0.4721,0.5279
9,0.4272,0.5728
10,0.4093,0.5907


Answer: As we can see in the table above, the start cell for Player 2 that give approximate equals odds for both players is the square number 5.

___
### Question 5
In a different attempt to change the odds of the game, instead of starting Player 2 on a different square, you decide to give Player 2 immunity to the first snake that they land on. What is the approximate probability that Player 1 wins now?

In [8]:
winners = []
for _ in range(0, runs):
    winner = play_round_immunity(players, board, moves)

    # Save the winner's name
    winners.append(winner.name)

    # Don't forget to reset the player current cell before playing again
    [p.reset() for p in players]

for p in players:
    print(p.name, winners.count(p.name) / len(winners))

Ale 0.43
Augusto 0.57


Answer: The probability of Player 1 win reduces approximately from 53% to 43%. Considering the low average of snakes in the game (3), the immunity of one snake is enough to change the scenario completely.